<a href="https://colab.research.google.com/github/omkar963/image_caption_generator/blob/main/Untitled4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
import string
import numpy as np
import pickle
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, Add
from tensorflow.keras.optimizers import Adam

print("✅ Imports loaded")

# ⚙️ Change these to your actual paths (if different)
DATASET_PATH = r"C:\Users\Omkar\.cache\kagglehub\datasets\adityajn105\flickr8k\versions\1"
CAPTIONS_FILE = os.path.join(DATASET_PATH, "captions.txt")
IMAGES_DIR = os.path.join(DATASET_PATH, "Images")

✅ Imports loaded


In [4]:
def load_captions(filename):
    mapping = {}
    with open(filename, 'r') as f:
        next(f)  # skip header
        for line in f:
            line = line.strip()
            if not line:
                continue
            img, caption = line.split(',', 1)
            caption = caption.lower()
            caption = caption.translate(str.maketrans('', '', string.punctuation))
            caption = "startseq " + caption + " endseq"
            mapping.setdefault(img, []).append(caption)
    return mapping

captions_dict = load_captions(CAPTIONS_FILE)
print(f"✅ Loaded {len(captions_dict)} images with captions")

✅ Loaded 8091 images with captions


In [5]:
def extract_features(images_dir, captions_dict, save_path="features.pkl", limit=8000):
    if os.path.exists(save_path):
        print("📂 Loading cached features...")
        with open(save_path, "rb") as f:
            return pickle.load(f)

    model = VGG16(weights="imagenet")
    model = Model(inputs=model.inputs, outputs=model.layers[-2].output)

    features = {}
    image_keys = list(captions_dict.keys())[:limit]  # limit to save memory
    for name in tqdm(image_keys, desc="Extracting features"):
        filename = os.path.join(images_dir, name)
        if not os.path.exists(filename):
            continue
        image = load_img(filename, target_size=(224, 224))
        image = img_to_array(image)
        image = np.expand_dims(image, axis=0)
        image = preprocess_input(image)
        feature = model.predict(image, verbose=0)
        features[name] = feature
    with open(save_path, "wb") as f:
        pickle.dump(features, f)
    print(f"✅ Extracted features for {len(features)} images")
    return features

features = extract_features(IMAGES_DIR, captions_dict)
print("Feature shape example:", next(iter(features.values())).shape)

📂 Loading cached features...
Feature shape example: (1, 4096)


In [6]:
all_captions = [c for caps in captions_dict.values() for c in caps]
tokenizer = Tokenizer(num_words=5000, oov_token="<unk>")
tokenizer.fit_on_texts(all_captions)

vocab_size = len(tokenizer.word_index) + 1
max_length = max(len(c.split()) for c in all_captions)
print(f"📊 Vocab size: {vocab_size}, Max length: {max_length}")

with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

📊 Vocab size: 8832, Max length: 38


In [7]:
# ============================================
# ⚙️ STEP 4 — DATA GENERATOR (MEMORY SAFE)
# ============================================
# Define the generator first
# ============================================
# ⚙️ STEP 4 — DATA GENERATOR (TF 2.15+ Compatible)
# ============================================
def data_generator(captions_dict, features, tokenizer, max_length, vocab_size):
    """Compatible generator for TensorFlow ≥2.15 / Keras 3"""
    while True:
        for key, desc_list in captions_dict.items():
            photo = features.get(key)
            if photo is None:
                continue
            for desc in desc_list:
                seq = tokenizer.texts_to_sequences([desc])[0]
                for i in range(1, len(seq)):
                    in_seq, out_seq = seq[:i], seq[i]
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    # yield tuple with float32 arrays (TF-safe)
                    yield ((photo.flatten().astype(np.float32),
                            in_seq.astype(np.float32)),
                           out_seq.astype(np.float32))

# Optional: wrap with tf.data.Dataset to improve performance
output_signature = (
    (
        tf.TensorSpec(shape=(4096,), dtype=tf.float32),
        tf.TensorSpec(shape=(max_length,), dtype=tf.float32)
    ),
    tf.TensorSpec(shape=(vocab_size,), dtype=tf.float32)
)

train_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(train_subset, features, tokenizer, max_length, vocab_size),
    output_signature=output_signature
).batch(32).prefetch(tf.data.AUTOTUNE)



In [8]:
def build_model(vocab_size, max_length):
    # Image feature extractor
    inputs1 = Input(shape=(4096,))
    fe1 = Dropout(0.5)(inputs1)
    fe2 = Dense(256, activation='relu')(fe1)

    # Sequence processor
    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
    se2 = Dropout(0.5)(se1)
    se3 = LSTM(256)(se2)

    # Decoder
    decoder1 = Add()([fe2, se3])
    decoder2 = Dense(256, activation='relu')(decoder1)
    outputs = Dense(vocab_size, activation='softmax')(decoder2)

    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001))
    return model

model = build_model(vocab_size, max_length)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 38)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer         │ (None, 4096)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 38, 256)   │  2,260,992 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 4096)      │          0 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 38, 256)   │          0 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, 38)        │          0 │ input_layer_1[0]… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 256)       │  1,048,832 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 256)       │    525,312 │ dropout_1[0][0],  │
│                     │                   │            │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 256)       │          0 │ dense[0][0],      │
│                     │                   │            │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 256)       │     65,792 │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 8832)      │  2,269,824 │ dense_1[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 6,170,752 (23.54 MB)

 Trainable params: 6,170,752 (23.54 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
train_subset = dict(list(captions_dict.items())[:1500])
steps_per_epoch = sum(len(c) for c in train_subset.values())

print("Training on subset for stability:", len(train_subset), "images")

history = model.fit(
    train_dataset,
    epochs=10,
    steps_per_epoch=steps_per_epoch,
    verbose=1
)

model.save("caption_model_evoastra.keras", save_format="keras")

print("✅ Model training complete")

Training on subset for stability: 1500 images
Epoch 1/10


7500/7500 ━━━━━━━━━━━━━━━━━━━━ 1031s 137ms/step - loss: 4.0319
Epoch 2/10
7500/7500 ━━━━━━━━━━━━━━━━━━━━ 1041s 139ms/step - loss: 3.2085
Epoch 3/10
7500/7500 ━━━━━━━━━━━━━━━━━━━━ 1025s 137ms/step - loss: 2.8950
Epoch 4/10
7500/7500 ━━━━━━━━━━━━━━━━━━━━ 821s 109ms/step - loss: 2.6977
Epoch 5/10
7500/7500 ━━━━━━━━━━━━━━━━━━━━ 815s 109ms/step - loss: 2.5432
Epoch 6/10
7500/7500 ━━━━━━━━━━━━━━━━━━━━ 822s 110ms/step - loss: 2.4304
Epoch 7/10
7500/7500 ━━━━━━━━━━━━━━━━━━━━ 810s 108ms/step - loss: 2.3421
Epoch 8/10
7500/7500 ━━━━━━━━━━━━━━━━━━━━ 906s 121ms/step - loss: 2.2579
Epoch 9/10
7500/7500 ━━━━━━━━━━━━━━━━━━━━ 1044s 139ms/step - loss: 2.2071
Epoch 10/10
7500/7500 ━━━━━━━━━━━━━━━━━━━━ 1068s 142ms/step - loss: 2.1620


✅ Model training complete


In [12]:
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

def generate_desc(model, tokenizer, photo, max_length):
    in_text = 'startseq'
    for _ in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length)
        yhat = model.predict([photo, sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = word_for_id(yhat, tokenizer)
        if word is None:
            break
        in_text += ' ' + word
        if word == 'endseq':
            break
    return in_text.replace('startseq', '').replace('endseq', '').strip()

In [13]:
def extract_feature_single(filename):
    model_vgg = VGG16(weights="imagenet")
    model_vgg = Model(inputs=model_vgg.inputs, outputs=model_vgg.layers[-2].output)
    image = load_img(filename, target_size=(224, 224))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = preprocess_input(image)
    feature = model_vgg.predict(image, verbose=0)
    return feature

# Use any local image path here
test_image = r"C:\Users\Omkar\Downloads\cutedog.jpeg"

photo = extract_feature_single(test_image)
caption = generate_desc(model, tokenizer, photo, max_length)
print("\n🖼️ Generated caption:\n", caption)


🖼️ Generated caption:
 a dog is running on a wooden bridge in the woods
